Somehow, this stuff is needed first, so pyterrier is loaded. Otherwiese, python wont load the whole module.

In [1]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client

ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


### Imports

In [2]:
from pyterrier import get_dataset, IterDictIndexer, BatchRetrieve
import pyterrier_doc2query

### Load the dataset

In [3]:
dataset1 = "irds:ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training"
dataset2 = "irds:vaswani"

In [4]:
pt_dataset = get_dataset(dataset2)

### Generate the index using "document expansion by query prediction"

In [5]:
# clear index folder first
import shutil
# shutil.rmtree("./data/index/")

In [ ]:
doc2query = pyterrier_doc2query.Doc2Query(append=True, verbose=True) # append generated queries to the orignal document text

## >> = custom operator
indexer = doc2query >> IterDictIndexer("./data/index")

In [5]:
doc2query = pyterrier_doc2query.Doc2Query(append=True, verbose=True, num_samples=1, batch_size=1) # append generated queries to the orignal document text

indexer = doc2query >> IterDictIndexer(
    # Store the index in the `index` directory.
    "./data/index",
    meta={'docno': 50, 'text': 4096},
    # If an index already exists there, then overwrite it.
    overwrite=True,
)

/usr/local/lib/python3.10/dist-packages/pyterrier_doc2query/__init__.py:44: UserWarning: consider setting fast_tokenizer=True; it speeds up inference considerably
  warn('consider setting fast_tokenizer=True; it speeds up inference considerably')
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
indexer = IterDictIndexer(
    # Store the index in the `index` directory.
    "./data/index",
    meta={'docno': 50, 'text': 4096},
    # If an index already exists there, then overwrite it.
    overwrite=True,
)

In [6]:
index = indexer.index(pt_dataset.get_corpus_iter())

vaswani documents:   0%|          | 0/11429 [00:00<?, ?it/s]

doc2query:  50%|█████     | 50/100 [07:09<07:09,  8.59s/d]


KeyboardInterrupt: 

### Build the Retriever

In [ ]:
bm25 = BatchRetrieve(index, wmodel="BM25")

In [ ]:
run = bm25(pt_dataset.get_topics('text'))